<a href="https://colab.research.google.com/github/ghgit1798/2021-financial-competition/blob/Guhyeon/%EC%8B%A0%ED%95%9C%EC%B9%B4%EB%93%9C_%EC%97%85%EC%A2%85%EB%8C%80%EB%B6%84%EB%A5%98%20%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import plotly.graph_objects as go
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [50]:
og = pd.read_csv('신한카드.csv', index_col=0, encoding='utf-8')

In [51]:
og.columns

Index(['기준년월', '광역시도명', '업종대분류', '업종중분류', '업종소분류', '신규가맹점수', '해지가맹점수',
       '매출가맹점수', '카드매출금액', '카드매출건수', '점당매출금액', '건당매출금액'],
      dtype='object')

In [52]:
# 업종대분류에 nan값과 음식업종이 2개로 분리되어 있다.
og['업종대분류'].unique()

array(['문화레져', '생활서비스', '음식', '일반유통', '전문서비스', '종합유통', nan, '기타', '음식 '],
      dtype=object)

In [53]:
og

,기준년월,광역시도명,업종대분류,업종중분류,업종소분류,신규가맹점수,해지가맹점수,매출가맹점수,카드매출금액,카드매출건수,점당매출금액,건당매출금액
0,201903,강원,문화레져,동물,동물농장,3,1,12,16036000,500,1336292,32071
1,201903,강원,문화레져,동물,애완동물/용품,5,1,190,283445000,7826,1491816,36218
2,201903,강원,문화레져,동물병원,병원-동물병원,0,1,88,341305000,4910,3878461,69512
3,201903,강원,문화레져,레져,골프장,0,1,80,3575639000,25016,44695491,142934
4,201903,강원,문화레져,레져,낚시터,1,1,3,748000,9,249333,83111
...,...,...,...,...,...,...,...,...,...,...,...,...
23887,202009,충북,종합유통,전용매장,복지매점,0,0,5,2199000,415,439877,5300
23888,202009,충북,종합유통,전용매장,사원전용매장,0,0,10,82163000,3971,8216280,20691
23889,202009,충북,종합유통,전용매장,연금매장,0,0,3,36451000,1602,12150400,22754
23890,202009,충북,종합유통,편의점,편의점,31,23,1726,10572495000,1365915,6125432,7740


In [54]:
# 업종분류가 되지 않은 종류가 72개이므로, 삭제할 지 레이블링을 해줄 지 결정해야 한다.
og.isnull().sum()

기준년월       0
광역시도명      0
업종대분류     72
업종중분류     72
업종소분류     72
신규가맹점수     0
해지가맹점수     0
매출가맹점수     0
카드매출금액     0
카드매출건수     0
점당매출금액     0
건당매출금액     0
dtype: int64

In [55]:
# 매출가맹점수가 1480개인 업종도 존재한다.
# 반면, 매출가맹점수가 0개인 것도 존재한다.
og[og['업종대분류'].isnull()]

,기준년월,광역시도명,업종대분류,업종중분류,업종소분류,신규가맹점수,해지가맹점수,매출가맹점수,카드매출금액,카드매출건수,점당매출금액,건당매출금액
334,201903,강원,NaN,NaN,NaN,0,0,492,1499526000,33382,3047818,44920
681,201903,경기,NaN,NaN,NaN,0,0,1480,26941555000,1685731,18203753,15982
1022,201903,경남,NaN,NaN,NaN,0,0,412,1092492000,21015,2651680,51986
1360,201903,경북,NaN,NaN,NaN,0,0,457,1822708000,34566,3988420,52731
1688,201903,광주,NaN,NaN,NaN,0,0,216,879589000,17898,4072172,49145
...,...,...,...,...,...,...,...,...,...,...,...,...
22550,202009,전남,NaN,NaN,NaN,0,0,0,0,0,0,0
22889,202009,전북,NaN,NaN,NaN,0,0,0,0,0,0,0
23214,202009,제주,NaN,NaN,NaN,0,0,0,0,0,0,0
23551,202009,충남,NaN,NaN,NaN,0,0,0,0,0,0,0


In [56]:
# 특정 지역에만 존재하는 가맹점이 있다. 이 산업을 분류하긴 어려우므로 삭제하자.
og = og[og['업종대분류'].notnull()]

In [57]:
# null값은 전부 삭제되었다.
og.isnull().sum()

기준년월      0
광역시도명     0
업종대분류     0
업종중분류     0
업종소분류     0
신규가맹점수    0
해지가맹점수    0
매출가맹점수    0
카드매출금액    0
카드매출건수    0
점당매출금액    0
건당매출금액    0
dtype: int64

In [58]:
# 다음은 음식 업종을 통합해야한다.
og['업종대분류'].unique()

array(['문화레져', '생활서비스', '음식', '일반유통', '전문서비스', '종합유통', '기타', '음식 '],
      dtype=object)

In [59]:
# 업종소분류가 요식배달인 경우 카테고리 분류가 잘못되어있다.
og[og['업종대분류']=='음식 ']

,기준년월,광역시도명,업종대분류,업종중분류,업종소분류,신규가맹점수,해지가맹점수,매출가맹점수,카드매출금액,카드매출건수,점당매출금액,건당매출금액
12395,202003,경기,음식,배달,요식배달,0,0,0,0,0,0,0
12738,202003,경남,음식,배달,요식배달,0,0,0,0,0,0,0
13080,202003,경북,음식,배달,요식배달,0,0,2,1554000,24,777000,64750
13416,202003,광주,음식,배달,요식배달,0,0,1,100000,1,100000,100000
14754,202003,부산,음식,배달,요식배달,0,0,1,93000,2,93000,46500
15103,202003,서울,음식,배달,요식배달,1,0,2,1270000,5,635000,254000
16046,202003,인천,음식,배달,요식배달,0,0,0,0,0,0,0
18413,202009,경기,음식,배달,요식배달,0,0,0,0,0,0,0
18765,202009,경남,음식,배달,요식배달,0,0,0,0,0,0,0
19110,202009,경북,음식,배달,요식배달,0,0,2,3370000,54,1685009,62408


In [78]:
# 업종대분류를 음식으로 변경해주자.
og.loc[og['업종대분류']=='음식 ', '업종대분류'] = '음식'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
og[og['업종대분류']=='음식 ']

,기준년월,광역시도명,업종대분류,업종중분류,업종소분류,신규가맹점수,해지가맹점수,매출가맹점수,카드매출금액,카드매출건수,점당매출금액,건당매출금액


In [81]:
# 업종대분류 카테고리 전처리가 완료되었다.
og['업종대분류'].unique()

array(['문화레져', '생활서비스', '음식', '일반유통', '전문서비스', '종합유통', '기타'], dtype=object)

In [82]:
og.to_csv('신한카드_업종대분류 전처리완료.csv', encoding='utf-8')